In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pathlib
from tensorflow import keras

In [2]:
kasus = np.array([1826.0, 2088.0, 2812.0, 2815.0, 2228.0, 1192.0, 1403.0,
                  1578.0, 987.0, 644.0, 415.0, 447.0], dtype=float)
zkasus =  np.array([x * 1/3500 for x in kasus], dtype=float)

doc1 = pd.read_csv("sample_data/data-curah-hujan-dan-hari-hujan-di-dki-jakarta-menurut-bulan-tahun-2014.csv")
doc2 = pd.read_csv("sample_data/data-kelembaban-udara-di-dki-jakarta-menurut-bulan-tahun-2014.csv")
doc3 = pd.read_csv("sample_data/data-suhu-udara-jakarta-menurut-bulan-tahun-2014.csv")

curah = np.array(doc1["curah_hujan"], dtype=float)
kelembaban = np.array(doc2["rata_rata"], dtype=float)
suhu = np.array(doc3["rata_rata"])
new_suhu = []
for x in suhu:
  new_suhu.append(x.replace(',','.'))
final_suhu = np.array(new_suhu, dtype=float)

zcurah =  np.array([x * 1/1200 for x in curah], dtype=float)
zkelembaban =  np.array([x * 1/100 for x in kelembaban], dtype=float)
zfinal_suhu =  np.array([x * 1/35 for x in final_suhu], dtype=float)

#print(zcurah)
#print(kelembaban)
#print(final_suhu)

merged_array = np.stack([zcurah, zkelembaban, zfinal_suhu], axis=1)
#print(merged_array)

#print(type(zcurah))

In [3]:
#doc1.head()
#doc2.head()
#doc3.head()

In [15]:
model = tf.keras.Sequential([
    keras.layers.Dense(3, input_dim=3, activation=keras.activations.sigmoid, use_bias=True),
    keras.layers.Dense(1, activation=keras.activations.relu, use_bias=True),
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(merged_array, zkasus, batch_size=16, epochs=1000)
model.summary()

Epoch 1/1000
1/1 [==============================] - 0s 269ms/step - loss: 0.0546
Epoch 2/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0542
Epoch 3/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0539
Epoch 4/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0535
Epoch 5/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0532
Epoch 6/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0529
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0527
Epoch 8/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.0525
Epoch 9/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0523
Epoch 10/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0521
Epoch 11/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0520
Epoch 12/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0519
Epoch 13/1000
1/1 [============================

In [16]:
the_input = np.expand_dims([1100.0/1200.0, 90.0/100.0, 27.0/35.0], axis=0)
print(the_input)
print(model.predict(the_input) * 3500)

[[0.91666667 0.9        0.77142857]]
[[2123.979]]


In [17]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: saved_model/1/assets


1500

In [18]:
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/1/ (stored 0%)
  adding: content/saved_model/1/assets/ (stored 0%)
  adding: content/saved_model/1/variables/ (stored 0%)
  adding: content/saved_model/1/variables/variables.index (deflated 58%)
  adding: content/saved_model/1/variables/variables.data-00000-of-00001 (deflated 69%)
  adding: content/saved_model/1/saved_model.pb (deflated 87%)


In [19]:
from google.colab import files
files.download("/content/saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>